# RSNA-MICCAI Brain Tumor Radiogenomic Classification

Thanks to previous great Notebooks.

1. [[TF]: 3D & 2D Model for Brain Tumor Classification][1]

2. [【Brain Tumor】EDA for starter(日本語version)][2]

3. [Efficientnet3D with one MRI type][3]

4. [🧠Brain Tumor 3D [Training]][4]

---

[1]: https://www.kaggle.com/ipythonx/tf-3d-2d-model-for-brain-tumor-classification

[2]: https://www.kaggle.com/chumajin/brain-tumor-eda-for-starter-version

[3]: https://www.kaggle.com/rluethy/efficientnet3d-with-one-mri-type

[4]: https://www.kaggle.com/ammarnassanalhajali/brain-tumor-3d-training/data

# 0. Settings

In [ ]:
# Import dependencies 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

import os, sys, glob, gc 
import math, re, random, time
from tqdm import tqdm 
import cv2, pydicom

from sklearn.model_selection import StratifiedKFold 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Params
config = {
    'data_path': '../input/rsna-miccai-brain-tumor-radiogenomic-classification',
    'model_path': '../input/keras-3d-efficientnet-imagenet-weights-b0b7/efficientnet3d_keras/efficientnet-b0_inp_channel_3_tch_0_top_False.h5',
    'input_path': '../input', 
    'output_path': './',
    'num_3d': 16,
    'img_size': 64,
    'n_gradients': 16,
    'nfolds': 5, 
    'batch_size': 16,
    'learning_rate': 1e-4,
    'num_epochs': 10
}

AUTO = tf.data.AUTOTUNE

# For reproducible results    
def seed_all(s):
    random.seed(s)
    np.random.seed(s)
    tf.random.set_seed(s)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['PYTHONHASHSEED'] = str(s) 
global_seed = 42
seed_all(global_seed)

input_modality = ["FLAIR", "T1w", "T1wCE", "T2w"]
modality_list = ["FLAIR", "T1w", "T2w"] 

train_folder = os.path.join(config['data_path'], 'train')
test_folder = os.path.join(config['data_path'], 'test')
sample_submission_path = os.path.join(config['data_path'], 'sample_submission.csv')

train_df = pd.read_csv(os.path.join(config['data_path'], 'train_labels.csv')); print(train_df.shape)
sample_df = pd.read_csv(sample_submission_path); print(sample_df.shape)
test_df = sample_df.copy(); print(test_df.shape)

# 1. EDA

## 1.1 Train DataFrame

In [ ]:
# Getting each folder paths of BraTS21ID

train_df['imfolder'] = ['{:05d}'.format(s) for s in train_df['BraTS21ID']]
train_df['path'] = [os.path.join(train_folder, s) for s in train_df['imfolder']]
train_df

In [ ]:
# Counting the files in FLAIR folder

#input_modality = ["FLAIR", "T1w", "T1wCE", "T2w"] 
input_modality = ["FLAIR"] 
for modality in input_modality:   
    modality_count = []
    for i in range(len(train_df)):
        sample_folder = train_df['path'].iloc[i]
        modality_folder = os.path.join(sample_folder, modality)
        if os.path.exists(modality_folder):
            modality_count.append(len(os.listdir(modality_folder)))
        else:
            modality_count.append(0)
        
    train_df[f'{modality}_count'] = modality_count    
    
train_df = train_df.query("FLAIR_count >= 16").reset_index()
    
train_df

In [ ]:
# k-fold (n=5) for cross-validation (I conducted hold-out validation in this notebook, though.)

skf = StratifiedKFold(n_splits=config['nfolds'], shuffle=True, random_state=global_seed)

for index, (train_index, val_index) in enumerate(skf.split(X=train_df.index, y=train_df.MGMT_value)):
    train_df.loc[val_index, 'fold'] = index
    
print(train_df.groupby(['fold', train_df.MGMT_value]).size())

## 1.2 Test DataFrame

In [ ]:
test_df['imfolder'] = ['{:05d}'.format(s) for s in test_df['BraTS21ID']]
test_df['path'] = [os.path.join(test_folder, s) for s in test_df['imfolder']]
test_df

In [ ]:
#input_modality = ["FLAIR", "T1w", "T1wCE", "T2w"] 
input_modality = ["FLAIR"] 

for modality in input_modality:   
    modality_count = []
    for i in range(len(test_df)):
        sample_folder = test_df['path'].iloc[i]
        modality_folder = os.path.join(sample_folder, modality)
        if os.path.exists(modality_folder):
            modality_count.append(len(os.listdir(modality_folder)))
        else:
            modality_count.append(0)
        
    test_df[f'{modality}_count'] = modality_count    
    
test_df = test_df.query("FLAIR_count >= 16").reset_index()

test_df

# 2. DataLoader

## 2.1 Train Dataset

In [ ]:
def get_img_path_3d(df, index, mri_type='FLAIR'):
    patient_id = df['BraTS21ID'][index]
    patient_path = df['path'][index]
    modality_path = os.path.join(patient_path, mri_type)
    total_img_num = df[f'{mri_type}_count'][index]
    
    files = sorted(glob.glob(f"{modality_path}/*.dcm"), 
                   key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    
    mid_num = total_img_num // 2
    num_3d2 = config['num_3d'] // 2
    start_idx = max(0, mid_num - num_3d2)
    end_idx = min(len(files), mid_num + num_3d2)
    
    target_file_paths = files[start_idx:end_idx]
    
    return target_file_paths

@tf.function
def preprocessing_img(img):
    #img = img / tf.math.reduce_max(img)
    img = tf.expand_dims(img, -1)
    img = tf.image.resize(img, [config['img_size'], config['img_size']])
    img = tf.expand_dims(img, -2)
    return img

    
class ImageGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, mri_type='FLAIR'):
        self.df = df
        self.mri_type = mri_type

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        paths = get_img_path_3d(self.df, index)
        img_list = []
        for path in paths:
            dicom = pydicom.read_file(path)
            img = dicom.pixel_array
            img = tf.convert_to_tensor(img, dtype=tf.float32)
            img = preprocessing_img(img)
            img_list.append(img)
        img_3d = tf.concat(img_list, axis=-2)
        return img_3d
    
    
def parse(x):
    result = tf.io.parse_tensor(x, out_type=tf.float32)
    result = tf.reshape(result, [config['img_size'], config['img_size'], config['num_3d'], 1])
    return result


def build_3d_train_dataloader(train_df, p_fold=0):
    p_train = train_df.query(f'fold != {p_fold}').reset_index(drop=True)
    p_valid = train_df.query(f'fold == {p_fold}').reset_index(drop=True)

    AUTOTUNE = tf.data.experimental.AUTOTUNE

    train_datasets = []
    for mode, df in zip(['train', 'valid'], [p_train, p_valid]):
        i_g = ImageGenerator(df)
        img_ds = tf.data.Dataset.from_generator(lambda: map(tuple, i_g),
                                                output_types=(tf.float32),
                                                output_shapes=(tf.TensorShape([config['img_size'], config['img_size'], config['num_3d'], 1])),
                                                 )
        
        serial_ds = img_ds.map(tf.io.serialize_tensor)

        if not os.path.exists(f'{mode}-{p_fold}-img.tfrec'):
            img_tfrec = tf.data.experimental.TFRecordWriter(f'{mode}-{p_fold}-img.tfrec')
            img_tfrec.write(serial_ds)
        serial_ds = tf.data.TFRecordDataset(f'{mode}-{p_fold}-img.tfrec')
        serial_ds = serial_ds.map(parse, num_parallel_calls=AUTOTUNE)

        labels = df['MGMT_value']
        label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int32))

        ds = tf.data.Dataset.zip((img_ds, label_ds))
        
        ds = ds.cache(filename=f'./cache.tf-{mode}-{p_fold}-data')
        if mode == 'train':
            train_count = len(df)
            ds = ds.shuffle(buffer_size=train_count)
        ds = ds.batch(config['batch_size'], drop_remainder=True)
        ds = ds.prefetch(buffer_size=AUTOTUNE)
        train_datasets.append(ds)

    return train_datasets

In [ ]:
# Building Dataset
p_fold = 0

train_datasets = build_3d_train_dataloader(train_df, p_fold=p_fold)
train_ds = train_datasets[0]
valid_ds = train_datasets[1]

for d, l in train_ds.take(1):
    print('Train Data shape: ', d.shape)
    print('Train Label shape: ', l.shape)
    
for d, l in valid_ds.take(1):
    print('Valid Data shape: ', d.shape)
    print('Valid Label shape: ', l.shape)

## 2.2 Test Dataset

In [ ]:
# TestDataset without Labels
def build_3d_test_dataloader(test_df):
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    i_g = ImageGenerator(test_df)
    img_ds = tf.data.Dataset.from_generator(lambda: map(tuple, i_g),
                                         output_types=(tf.float32),
                                         output_shapes=(tf.TensorShape([config['img_size'], config['img_size'], config['num_3d'], 1])),
                                                 )
    serial_ds = img_ds.map(tf.io.serialize_tensor)

    if not os.path.exists('test-img.tfrec'):
        img_tfrec = tf.data.experimental.TFRecordWriter('test-img.tfrec')
        img_tfrec.write(serial_ds)
    serial_ds = tf.data.TFRecordDataset('test-img.tfrec')
    test_ds = serial_ds.map(parse, num_parallel_calls=AUTOTUNE)

    test_ds = test_ds.cache(filename='./cache.tf-test-data')
    test_ds = test_ds.batch(config['batch_size'], drop_remainder=False)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

    return test_ds

In [ ]:
test_ds = build_3d_test_dataloader(test_df)

for d in test_ds.take(1):
    print('Test Data shape: ', d.shape)

# 3. Train

## 3.1 Model

In [ ]:
def get_3d_model(width=config['img_size'], height=config['img_size'], depth=config['num_3d']):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))
    
    x = layers.Conv3D(filters=32, kernel_size=3, padding='same', activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=32, kernel_size=3, padding='same', activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=64, kernel_size=3, padding='same', activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.Conv3D(filters=128, kernel_size=3, padding='same', activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.02)(x)

    x = layers.Conv3D(filters=256, kernel_size=3, padding='same', activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.03)(x)

    x = layers.Conv3D(filters=512, kernel_size=3, padding='same', activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.04)(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=1024, activation="relu")(x)
    x = layers.Dropout(0.08)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    model = keras.Model(inputs, outputs, name="3dcnn")

    return model


model = get_3d_model()
model.summary()

## Save Model here

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

In [ ]:
class BrainTumorModel3D(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)        
        self.cnn = get_3d_model()
        
    @tf.function
    def call(self, input_tensor, training=False, **kwargs):
        x = self.cnn(input_tensor)
        return x
    
    def build_graph(self, raw_shape):
        x = tf.keras.layers.Input(shape=raw_shape)
        return tf.keras.Model(inputs=[x], outputs=self.call(x))


if tf.test.is_gpu_available():
    device_name = tf.test.gpu_device_name()
else:
    device_name = 'cpu:0'

with tf.device(device_name):
    model = BrainTumorModel3D()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

train_acc_metric = tf.keras.metrics.BinaryAccuracy()
val_acc_metric = tf.keras.metrics.BinaryAccuracy()

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=config['output_path'],
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

@tf.function
def train_step(x, y):
    
    with tf.GradientTape() as tape:
        pred_y = model(x, training=True)
        train_loss = loss_fn(y, pred_y)
        
    grads = tape.gradient(train_loss, model.trainable_weights)
    
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    train_acc_metric.update_state(y_true=y, y_pred=pred_y)
    
    return train_loss


@tf.function
def valid_step(x, y):
    pred_y = model(x, training=False)
    val_loss = loss_fn(y, pred_y)
    
    val_acc_metric.update_state(y_true=y, y_pred=pred_y)
    
    return val_loss

In [ ]:
train_history = []
valid_history = []

for epoch in range(config['num_epochs']):
    t = time.time()
    
    train_loss_list = []
    val_loss_list = []
    
    for x, y in train_ds:
        train_batch_loss = train_step(x, y)
        train_loss_list.append(train_batch_loss)
        
    for x, y in valid_ds:
        val_batch_loss = valid_step(x, y)
        val_loss_list.append(val_batch_loss)
        
    train_loss = sum(train_loss_list) / len(train_loss_list)
    val_loss = sum(val_loss_list) / len(val_loss_list)
    
    train_acc = train_acc_metric.result()
    val_acc = val_acc_metric.result()
    
    train_history.append(train_loss)
    valid_history.append(val_loss)
    
    template = 'ETA: {} -- epoch: {}, loss: {}  acc: {}  val_loss: {}  val_acc: {}\n'
    print(template.format(
                   round((time.time() -  t) / 60, 2), epoch+1,
                   (train_loss, '.3f'), (train_acc, '.3f'),
                   (val_loss, '.3f'), (val_acc, '.3f'))
         )
    
    train_acc_metric.reset_states()
    val_acc_metric.reset_states()

# 4. Prediction 

In [ ]:
proba = model.predict(test_ds, batch_size=config['batch_size'], verbose=1)
proba

In [ ]:
test_df['prediction'] = proba
sample_df['MGMT_value'] = test_df['prediction']
sample_df

In [ ]:
sample_df.to_csv("submission.csv", index=False)

# cf. Gradient Accumulation Model

In [ ]:
class GradAcumModel(tf.keras.Model):
    def __init__(self, model, n_gradients=config['n_gradients'], *args, **kwargs):
        super(GradAcumModel, self).__init__(*args, **kwargs)
        self.model = model
        self.n_gradients = tf.constant(n_gradients, dtype=tf.int32)
        self.n_acum_step = tf.Variable(0, dtype=tf.int32, trainable=False)
        self.gradient_accumulation = [tf.Variable(tf.zeros_like(v, dtype=tf.float32),
                                                  trainable=False)
                                       for v in self.model.trainable_variables]

    @tf.function
    def train_step(self, data):
        self.n_acum_step.assign_add(1)
        images, labels = data

        with tf.GradientTape() as tape:
            predictions = self.model(images, training=True)
            loss = self.compiled_loss(labels, predictions)

        gradients = tape.gradient(loss, self.model.trainable_variables)

        for i in range(len(self.gradient_accumulation)):
            self.gradient_accumulation[i].assign_add(gradients[i])

        # If n_acum_step reach the n_gradients then we apply accumulated gradients -
        # - to update the variables otherwise do nothing
        tf.cond(tf.equal(self.n_acum_step, self.n_gradients),
                self.apply_accu_gradients, lambda: None)
        
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def apply_accu_gradients(self):
        self.optimizer.apply_gradients(zip(self.gradient_accumulation,
                                           self.model.trainable_variables))
        
        # Reset
        self.n_acum_step.assign(0)
        for i in range(len(self.gradient_accumulation)):
            self.gradient_accumulation[i].assign(
                tf.zeros_like(self.model.trainable_variables[i], dtype=tf.float32)
            )

    @tf.function
    def test_step(self, data):
        images, labels = data

        predictions = self.model(images, training=False)
        loss = self.compiled_loss(labels, predictions)
        self.compiled_metrics.update_state(labels, predictions)
        return {m.name: m.result() for m in self.metrics}

    def call(self, inputs, *args, **kwargs):
        return self.model(inputs)

with tf.device(device_name):
    grad_acum_model = GradAcumModel(model, n_gradients=4)

In [ ]:
grad_acum_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(name='acc'), ],
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=config['output_path'],
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Train the model
#grad_acum_model.fit(
#    train_ds,
#    epochs=config['num_epochs'],
#    validation_data=valid_ds,
#    callbacks=[model_checkpoint],
#    verbose=1
#)